<a href="https://colab.research.google.com/github/tonystz/cuda/blob/main/poc/gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Get code

In [ ]:
!git clone https://github.com/tonystz/cuda.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python cuda/poc/raw.py


###CPU

In [11]:
!python cuda/poc/cpu.py

[step1]reload data: 1.4249060153961182 (1141458,) |S44 1141458 [b'::1-200' b'::1-200' b'::1-404' ... b'180.163.28.55-200'
 b'180.163.28.55-200' b'180.163.28.55-200']
[step2]404 data filter: 0.29528236389160156 202448
[step3]check result: 0.08500218391418457 ['216.244.66.200', '217.174.192.132', '66.249.79.234', '66.249.79.238', '66.249.79.236', '23.96.16.211', '47.101.154.149', '47.101.149.21', '38.54.24.118']


###GPU initialize

In [ ]:
!pip install pycuda # install cuda
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np

###GPU

In [12]:
a=np.loadtxt('pre.log', delimiter=' ', dtype=np.string_,converters={0:lambda v:v+b'\0'})
print(a.shape,a.size,a)

(1141458,) 1141458 [b'::1-200' b'::1-200' b'::1-404' ... b'180.163.28.55-200'
 b'180.163.28.55-200' b'180.163.28.55-200']


In [6]:
print(a[0])

b'::1-200'


In [13]:
na=np.delete(a,0)
na.reshape((479,2383))

array([[b'::1-200', b'::1-404', b'::1-200', ..., b'195.242.110.253-200',
        b'110.249.202.11-200', b'110.249.201.69-200'],
       [b'110.249.201.206-200', b'207.46.13.23-200',
        b'110.249.201.14-200', ..., b'223.104.68.50-200',
        b'223.104.68.50-200', b'223.104.68.50-200'],
       [b'223.104.68.50-200', b'223.104.68.50-200', b'223.104.68.50-200',
        ..., b'125.84.82.12-200', b'125.84.82.12-200',
        b'125.84.82.12-200'],
       ...,
       [b'140.250.200.255-404', b'140.250.200.255-404',
        b'52.167.144.32-200', ..., b'180.164.68.86-200',
        b'180.164.68.86-200', b'180.164.68.86-200'],
       [b'180.164.68.86-200', b'180.164.68.86-200', b'180.164.68.86-200',
        ..., b'36.46.86.9-200', b'36.46.86.9-200', b'36.46.86.9-200'],
       [b'36.46.86.9-200', b'36.46.86.9-200', b'36.46.86.9-200', ...,
        b'180.163.28.55-200', b'180.163.28.55-200', b'180.163.28.55-200']],
      dtype='|S44')

In [ ]:
s= np.array(['180.163.28.55-200\0','edfg\0','stz12345678b\0','b1\0','k2\0','hellworld3\0'], dtype=np.string_)

s=s.reshape(2,3)
print('shape:',s.shape,s.size)
print(s,s.dtype)

shape: (2, 3) 6
[[b'180.163.28.55-200' b'edfg' b'stz12345678b']
 [b'b1' b'k2' b'hellworld3']] |S18


In [85]:
%%writefile gpu.py
#!python 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

mod = SourceModule("""

 const int COL_NUM=2383;
 const int ROW_NUM=479;
 const int LOG_LEN=44;
    typedef struct{
      char ipAddr[LOG_LEN];
      int cnt;
    }ST_IPAddr;
    __device__ void mystrcpy(char *out,char *in){
      int i=0;
      for(i=0;in[i]!=0;i++){
        out[i] = in[i];
      }
      out[i]=0;
    }
    __device__  int find_404_ipaddr(char*in_gpu,char outAddr[],int rowStartIndex ,int strStartIndex){
        int index=0;
        for(int i = 0; i<LOG_LEN; i++){
            index = strStartIndex+i+rowStartIndex;
            if (in_gpu[index]=='-'){
              if (in_gpu[index+1] == '4' && in_gpu[index+2] == '0' && in_gpu[index+3] == '4'){
                 outAddr[i]=0;
                 return 1;
              }
            }
            outAddr[i]=in_gpu[index];
        }
        return 0;
    }
   
    __device__ int strEqua(char *s1,char *s2){
      for(int i=0;i<LOG_LEN;i++){
        if (s1[i] != s2[i]){
          return 0;
        }
      }
      return 1;
    }
    __device__ int pushDedup(ST_IPAddr*dedup,char *ipAddr, int dedup_cnt){
      for(int c=0;c<dedup_cnt;c++){
        if(strEqua(dedup[c].ipAddr,ipAddr)){
          dedup[c].cnt ++;
        }else{
          mystrcpy(dedup[dedup_cnt].ipAddr,ipAddr);
          dedup[dedup_cnt].cnt = 0;
        }
      }
    }
    __global__ void check_log(char*in_gpu,char* out_gpu)
    { 
      ST_IPAddr dedup[COL_NUM];
      int dedup_cnt=0;

      int idx = threadIdx.x + blockIdx.x * blockDim.x;

      //printf("index: threadIdx.x=%d blockIdx.x=%d  blockDim.x=%d \\n",threadIdx.x,blockIdx.x,blockDim.x);
     // printf("thread id: [%d]  get str:%s  strlen=%d \\n",idx,in_gpu+LOG_LEN+1, LOG_LEN);

      // string len is 6
      char sub[LOG_LEN+1];
      int strStartIndex=0;//string start locaion
      int rowOffset=COL_NUM*LOG_LEN;
      if(idx > ROW_NUM){
        printf("[ERROR]idx=%d, rowCnt=%d, we want one thread process all one rows'data .please tune",idx,ROW_NUM);
      }
      
      int rowStartIndex=rowOffset*idx;
      out_gpu[rowStartIndex]=0;
      //int all_find_iadd_in_row=0;
      for(int c=0;c<COL_NUM;c++){
        strStartIndex=c*LOG_LEN;
        if (find_404_ipaddr(in_gpu,sub,rowStartIndex,strStartIndex)){
          //printf("thread[%d] rowStartIndex=%d, substring:%s strStartIndex %d\\n",idx,rowStartIndex,sub,strStartIndex);
          
          //copy 404 to out_gpu
          //mystrcpy(out_gpu+rowStartIndex+dedup_cnt*LOG_LEN,sub);
          //mystrcpy(dedup[c].ipAddr,sub);
          pushDedup(dedup,sub,dedup_cnt);
          //printf("rowStartIndex=%d,c=%d  add=\%s\\n",rowStartIndex,c,dedup[c].ipAddr);
          dedup_cnt++;
        }
      }

      __syncthreads();
      for(int c=0;c<dedup_cnt;c++){
        
         mystrcpy(out_gpu+rowStartIndex+LOG_LEN*c,dedup[c].ipAddr);
         //printf("thread[%d][%d] addr:%s = %d\\n",rowStartIndex,c,dedup[c].ipAddr,dedup[c].cnt);
      }
      //printf("thread[%d] addrCnt:%d\\n",idx,dedup_cnt);
      __syncthreads();
    }
    """)

check_log = mod.get_function("check_log")

a=np.loadtxt('pre.log', delimiter=' ', dtype=np.string_,converters={0:lambda v:v+b'\0'})
na=np.delete(a,0)
NUM_ROW=479
NUM_COL=2383
na.reshape((479,2383))
print('shape:',na.shape,na.size)
print(na,na.data)

na_gpu = gpuarray.to_gpu(na)
out_gpu = gpuarray.empty_like(na_gpu)
check_log(na_gpu,out_gpu,block=(NUM_ROW,1,1),grid=(1,1,1))

import sys
np.set_printoptions(threshold=sys.maxsize)
print('out_gpu:',out_gpu.get())



Overwriting gpu.py


In [86]:
!python gpu.py

shape: (1141457,) 1141457
[b'::1-200' b'::1-404' b'::1-200' ... b'180.163.28.55-200'
 b'180.163.28.55-200' b'180.163.28.55-200'] <memory at 0x7f1766ce9700>
out_gpu: Traceback (most recent call last):
  File "/content/gpu.py", line 119, in <module>
    print('out_gpu:',out_gpu.get())
  File "/usr/local/lib/python3.9/dist-packages/numpy/core/arrayprint.py", line 1592, in _array_str_implementation
    return array2string(a, max_line_width, precision, suppress_small, ' ', "")
  File "/usr/local/lib/python3.9/dist-packages/numpy/core/arrayprint.py", line 736, in array2string
    return _array2string(a, options, separator, prefix)
  File "/usr/local/lib/python3.9/dist-packages/numpy/core/arrayprint.py", line 513, in wrapper
    return f(self, *args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/numpy/core/arrayprint.py", line 546, in _array2string
    lst = _formatArray(a, format_function, options['linewidth'],
  File "/usr/local/lib/python3.9/dist-packages/numpy/core/arrayprint.p

In [ ]:
!ls -l  /tmp/
!nvcc --cubin -arch sm_75 /tmp/tmp6h7ouqd_/kernel.cu